In [1]:
pip install opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import os

In [3]:
# Load images from directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir('Project'):
        path = os.path.join(folder, filename)
        img = cv2.imread(path, 0)  # Load in grayscale
        if img is not None:
            images.append(img)
    return images

In [4]:
# Feature extractor using ORB
def extract_features(img):
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(img, None)
    return keypoints, descriptors

In [5]:
# Compare descriptors using Brute Force Matcher
def match_features(desc1, desc2):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    if desc1 is None or desc2 is None:
        return 0
    matches = bf.match(desc1, desc2)
    return len(matches)

In [6]:
# Load training images
real_folder = 'Project/real_notes'
fake_folder = 'Project/fake_notes'
real_images = load_images_from_folder(real_folder)
fake_images = load_images_from_folder(fake_folder)

In [7]:
# Extract features for training data
real_features = [extract_features(img)[1] for img in real_images]
fake_features = [extract_features(img)[1] for img in fake_images]

In [8]:
# Load test image
test_img = cv2.imread('Project/test_note.jpg', 0)
_, test_des = extract_features(test_img)

In [9]:
# Match against real and fake notes
real_scores = [match_features(test_des, desc) for desc in real_features]
fake_scores = [match_features(test_des, desc) for desc in fake_features]

In [10]:
# Safe average calculation
real_avg = np.mean(real_scores) if real_scores else 0
fake_avg = np.mean(fake_scores) if fake_scores else 0

print("Real Avg Matches:", real_avg)
print("Fake Avg Matches:", fake_avg)

Real Avg Matches: 0
Fake Avg Matches: 0


In [11]:
if real_avg > fake_avg:
    print("The note is likely REAL.")
else:
    print("The note is likely FAKE.")

The note is likely FAKE.


In [12]:
keypoints, _ = extract_features(test_img)
img_with_kp = cv2.drawKeypoints(test_img, keypoints, None, color=(0, 255, 0))
cv2.imshow("Test Note Keypoints", img_with_kp)
cv2.waitKey(0)
cv2.destroyAllWindows()